### BEST PRACTICE : Use class composition instead of a complex nested internal state dictionary

When building an interface it is better to implement multiple simple layers than a complex nested state dictionary. Indeed as the interface evolve, it is getting harder to access relevant data as the state dictionary keeps getting deeper. It becomes a nightmare to debug. Instead, use simple layers.

In [3]:
# grade --> subject --> student --> book

In [4]:
from collections import namedtuple
Grade = namedtuple("Grade", ("score", "weight"))

In [5]:
Grade(85, 0.6)

Grade(score=85, weight=0.6)

In [6]:
from collections import defaultdict
import reprlib

# create a repr object to limit the number of arguments displayed with __repr__
subjectRepr = reprlib.Repr()
subjectRepr.maxlist = 3

class Subject:
    def __init__(self):
        self._grades = []

    def report_grade(self, score, weight):
        self._grades.append(Grade(score, weight))

    def __repr__(self):
        cls_name = type(self).__name__
        return f"{cls_name}({subjectRepr.repr(self._grades)})"

    @property # make it a read only property
    def average_grade(self):
        return sum(gr.score*gr.weight for gr in self._grades)/sum(gr.weight for gr in self._grades)

In [7]:
maths = Subject()
maths.report_grade(85,0.25)
maths.report_grade(98,0.55)
maths.report_grade(100,0.45)
maths

Subject([Grade(score=85, weight=0.25), Grade(score=98, weight=0.55), Grade(score=100, weight=0.45)])

In [8]:
maths.average_grade

96.12

In [9]:
class Student:

    def __init__(self):
        self._subjects = defaultdict(Subject) # return Subject object if key does not exists

    def get_subject(self, name):
        return self._subjects[name] # Instantiate Subject() if name not in dict otherise points to name value

    def __repr__(self):
        cls_name = type(self).__name__
        return f"{cls_name}({self._subjects})"

    @property
    def average_grade(self):
        return sum(subject.average_grade for subject in self._subjects.values())/len(self._subjects)

In [10]:
Nicolas = Student()
Nicolas

Student(defaultdict(<class '__main__.Subject'>, {}))

In [11]:
maths = Nicolas.get_subject('maths') # retrieve subject 
maths.report_grade(85, 0.25)
maths.report_grade(88, 0.25)

Nicolas.average_grade

86.5

In [12]:
Nicolas.__dict__

{'_subjects': defaultdict(__main__.Subject,
             {'maths': Subject([Grade(score=85, weight=0.25), Grade(score=88, weight=0.25)])})}

In [13]:
class Gradebook:
    def __init__(self):
        self._elements = defaultdict(Student)

    def get_student(self, name):
        return self._elements[name]

In [14]:
book = Gradebook()
book.get_student('Nicolas')

Student(defaultdict(<class '__main__.Subject'>, {}))

In [15]:
book = Gradebook()
albert = book.get_student('Albert Einstein')
math = albert.get_subject('Math')
math.report_grade(75, 0.05)
math.report_grade(65, 0.15)
math.report_grade(70, 0.80)
gym = albert.get_subject('Gym')
gym.report_grade(100, 0.40)
gym.report_grade(85, 0.60)
print(albert.average_grade)

80.25
